In [181]:
from collections import namedtuple
import numpy as np


with open('inputs/day15.txt') as input:
    everything = ''.join(input.readlines())

def move_string_to_move(move_string):
    match move_string:
        case '<':
            return np.array([0, -1])
        case '>':
            return np.array([0, 1])
        case '^':
            return np.array([-1, 0])
        case 'v':
            return np.array([1, 0])


grid_string, moves_string = everything.split('\n\n')
initial_grid = np.array([list(x) for x in grid_string.split('\n')])
moves = list(move_string_to_move(m) for m in moves_string.replace('\n', ''))
State = namedtuple('State', ('robot', 'grid'))

def step(state, move):
    curr = state.robot
    assert state.grid[*curr] == '@'
    if state.grid[*(curr + move)] == '.':
        state.grid[*(curr + move)] = '@'
        state.grid[*curr] = '.'
        return State(robot=curr + move, grid=state.grid)
    k = 1
    while state.grid[*(curr+k*move)] == 'O':
        k += 1
    if state.grid[*(curr+k*move)] == '#':
        return state
    state.grid[*(curr+k*move)] = 'O'
    state.grid[*(curr+move)] = '@'
    state.grid[*(curr)] = '.'
    return State(robot=curr + move, grid=state.grid)


def print_state(state):
    for i in range(len(state.grid)):
        for j in range(len(state.grid[i])):
            print(state.grid[i, j], end='')
        print()


def gps_sum(state):
    counter = 0
    for i in range(len(state.grid)):
        for j in range(len(state.grid[i])):
            if state.grid[i, j] == 'O':
                counter += 100 * i + j
    return counter

initial_robot = np.array(np.where(initial_grid == '@')).flatten()
initial_state = State(robot=initial_robot, grid=initial_grid.copy())
state = initial_state
for move in moves:
    state = step(state, move)
print_state(state)
gps_sum(state)

##################################################
#OO#..O#......#.....OOO##.OO.....OO..........#O..#
#...OOO...O.OO.O...OO..OO.#...O...OOO#O..#.#..O.##
#..OOO...O.O...O...OO...#OOO..O...OO#.#.OO.O...O.#
#O..OO.O..#.....O...#OO....#...#O.OO#O...O.O.#...#
#.....O...OOOO...........OOO......OOOO#...#O.....#
#....O....O.O.O#OO#.....#O.......O..##..#......#.#
#..#.O....OO.O..O..O......OOO...OO.O.........OOOO#
#.....#...O##..O..#O..........O.......O...#..OO..#
#......OOOO...O.....O.......O#........O.......OO##
#.....O#OO...O........................O.......#O.#
#.....OO.#O..O##O.OOO.............OOO.O..O.....OO#
#......O..O#OO..O..OO#...OO.......OO.#O.#O#..#...#
#..#....#OO.O..O...O.....#OOOO..O......##......O.#
#......OO.O.#.......OOO...O#OOOO..#....##...O...O#
#O.....#.#O#O.......O#....#O##OO..#.O..OO....O..O#
#O#.....#O#OO........O....OOOO#.....O....O.OO..O##
#OO........O.........OO##O#...OO...O..O.....O...O#
#OO..............OO..O#O...O#.#.##....OO...#O.O.O#
#OO#.....O.......#..OOOO.#.O...

1538871

In [182]:
def transform_grid(grid):
    new_grid = np.full((grid.shape[0], 2 * grid.shape[1]), '.', dtype=grid.dtype)
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            match grid[i, j]:
                case '#':
                    new_grid[i, 2*j:2*j+2] = ['#', '#']
                case 'O':
                    new_grid[i, 2*j:2*j+2] = ['[', ']']
                # case '.':
                #     new_grid[i, 2*j:2*j+2] = []
                case '@':
                    new_grid[i, 2*j:2*j+2] = ['@', '.']
    return new_grid

def get_box_tiles(grid, i, j, dir, s):
    if (i, j) in s:
        return
    if grid[i, j] not in ['[', ']']:
        return
    s.add((i, j))
    if grid[i, j] == '[':
        get_box_tiles(grid, i, j + 1, dir, s)
    else:
        get_box_tiles(grid, i, j - 1, dir, s)
    get_box_tiles(grid, i + dir, j, dir, s)

def valid_move(grid, tiles, move):
    for tile in tiles:
        if grid[*(tile + move)] == '#':
            return False
    return True

def step(state, move):
    curr = state.robot
    assert state.grid[*curr] == '@'
    if state.grid[*(curr + move)] == '.':
        state.grid[*(curr + move)] = '@'
        state.grid[*curr] = '.'
        return State(robot=curr + move, grid=grid)
    if state.grid[*(curr + move)] == '#':
        return state
    # easy part first: left or right
    if move[0] == 0:
        k = 1
        while state.grid[*(curr+k*move)] in ['[', ']']:
            k += 1
        if state.grid[*(curr+k*move)] == '#':
            return state
        r1, r2 = min(curr[1], curr[1]+(k-1)*move[1]), max(curr[1], curr[1]+(k-1)*move[1])
        state.grid[curr[0], r1+move[1]:r2+move[1]+1] = state.grid[curr[0], r1:r2+1]
        state.grid[*(curr)] = '.'
        return State(robot=curr + move, grid=state.grid)
    else:
        tiles = set()
        get_box_tiles(state.grid, curr[0] + move[0], curr[1], move[0], tiles)
        if not valid_move(state.grid, tiles, move):
            return state
        for i, j in sorted(tiles, key=lambda t: t[0] if move[0] == -1 else -t[0]):
            state.grid[i + move[0], j] = state.grid[i, j]
            state.grid[i, j] = '.'
        state.grid[*(curr+move)] = '@'
        state.grid[*(curr)] = '.'
        return State(robot=curr + move, grid=state.grid)

def gps_sum(state):
    counter = 0
    for i in range(len(state.grid)):
        for j in range(len(state.grid[i])):
            if state.grid[i, j] == '[':
                counter += 100 * i + j

    return counter
    
grid = transform_grid(initial_grid)
initial_robot = np.array(np.where(grid == '@')).flatten()
initial_state = State(robot=initial_robot, grid=grid[:])
state = initial_state
for move in moves[:]:
    state = step(state, move)
print_state(state)
gps_sum(state)

####################################################################################################
##[][]##....[]##....[]....[]##[][][][][]....[]####[]..[]....[]....[]..[]..[]..............##......##
##......[]..[]......[]....[][][][][]............[][]##......[]......[][][]##[]....##..##....[]..####
##....[]..[]...................[][]...[][]......##...[][]...[][]....[][]##..##.....[].[]....[][]..##
##..........[]......##........[]....[][]##............##......##........##[]..........[]..##......##
##.........................[].......[].[]........[]....[]......[].........[]##...[].##[]..........##
##....[][]..............[]....##....##.....[]...##.[].............[]....####[]..##....[]..[][]##..##
##....##......[]..[][][][]..........[]......[].[][]........[]...[].[].......[]........[]....[][][]##
##......[]..##.[][]...####..........##......[]...........[].[]..[][]........[]......##[].[]...[]..##
##..........[][]..[]..[][].........[].....................##......[]................[]...[]

1543338